In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import cv2
import numpy as np
import os

input_root = "/content/drive/MyDrive/clean_data_MRI/BPD"
output_root = "/content/drive/MyDrive/clean_data_MRI/brain region"
os.makedirs(output_root, exist_ok=True)

def process_image(img_path, out_path):
    img = cv2.imread(img_path)
    if img is None:
        return False

    h, w, _ = img.shape
    x1, y1 = 875, 512
    cv2.rectangle(img, (x1, y1), (w, h), (50, 50, 50), -1)

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([25, 80, 80])
    upper_yellow = np.array([36, 200, 200])
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    points = np.column_stack(np.where(mask > 0))
    points = np.flip(points, axis=1)

    if len(points) > 5:
        ellipse = cv2.fitEllipse(points)
        mask_ellipse = np.zeros(img.shape[:2], dtype=np.uint8)
        cv2.ellipse(mask_ellipse, ellipse, 255, -1)

        b, g, r = cv2.split(img)
        alpha = mask_ellipse
        rgba = cv2.merge([b, g, r, alpha])

        hsv2 = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        lower_yellow = np.array([25, 80, 80])
        upper_yellow = np.array([40, 255, 255])
        mask_yellow = cv2.inRange(hsv2, lower_yellow, upper_yellow)

        lower_green = np.array([40, 40, 40])
        upper_green = np.array([80, 255, 255])
        mask_green = cv2.inRange(hsv2, lower_green, upper_green)

        lower_red1 = np.array([0, 80, 80])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160, 80, 80])
        upper_red2 = np.array([180, 255, 255])
        mask_red1 = cv2.inRange(hsv2, lower_red1, upper_red1)
        mask_red2 = cv2.inRange(hsv2, lower_red2, upper_red2)
        mask_red = cv2.bitwise_or(mask_red1, mask_red2)

        mask_noise = cv2.bitwise_or(mask_yellow, mask_green)
        mask_noise = cv2.bitwise_or(mask_noise, mask_red)
        mask_noise = cv2.bitwise_and(mask_noise, alpha)

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        inpainted_gray_ns = cv2.inpaint(gray, mask_noise, inpaintRadius=3, flags=cv2.INPAINT_NS)
        median_full = cv2.medianBlur(inpainted_gray_ns, 7)

        result_rgba = cv2.merge([median_full, median_full, median_full, alpha])

        cv2.imwrite(out_path, result_rgba)
        return True
    else:
        return False


for root, dirs, files in os.walk(input_root):
    rel_path = os.path.relpath(root, input_root)
    out_dir = os.path.join(output_root, rel_path)
    os.makedirs(out_dir, exist_ok=True)

    if files:
        print(f"📂 Đang xử lý folder: {rel_path}")

    for file in files:
        if file.lower().endswith((".png", ".jpg", ".jpeg")):
            in_path = os.path.join(root, file)
            out_path = os.path.join(out_dir, os.path.splitext(file)[0] + "_brain.png")
            process_image(in_path, out_path)


📂 Đang xử lý folder: BPD binh thuong tu 14 den 2808
📂 Đang xử lý folder: BPD dau dai
📂 Đang xử lý folder: BPD nang dam roi mach mac nao that ben
📂 Đang xử lý folder: BPD nao truoc khong phan chia
📂 Đang xử lý folder: BPD xuat huyet noi so
📂 Đang xử lý folder: BPD canxi hoa doi thi
📂 Đang xử lý folder: BPD thung lung sylvius khong tuong xung tuoi thai
📂 Đang xử lý folder: BPD dan ho sau
📂 Đang xử lý folder: BPD dan nao that
📂 Đang xử lý folder: BPD bat san the chai
